# 1편: 이미지의 디지털 표현과 신경망 첫걸음

이 노트북에서는 다음을 학습합니다:
- 이미지가 컴퓨터에서 어떻게 숫자 배열로 표현되는지
- RGB 채널의 의미와 분리 방법
- PyTorch 텐서와 Linear 레이어 기초
- 학습의 핵심 사이클: Forward → Loss → Backward → Step

## 1. 이미지는 숫자다

컴퓨터가 보는 이미지는 픽셀 값으로 이루어진 3차원 배열입니다.
- **height**: 세로 픽셀 수
- **width**: 가로 픽셀 수
- **channels**: 색상 채널 수 (RGB = 3)

In [ ]:
from PIL import Image
import numpy as np
import urllib.request
import os

# 샘플 이미지 다운로드
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/280px-PNG_transparency_demonstration_1.png"
img_path = "sample_image.png"
if not os.path.exists(img_path):
    urllib.request.urlretrieve(img_url, img_path)

img = Image.open(img_path).convert("RGB")
img

In [ ]:
np_img = np.array(img)
print(f"이미지 shape: {np_img.shape}")
print(f"  → 높이={np_img.shape[0]}px, 너비={np_img.shape[1]}px, 채널={np_img.shape[2]}(RGB)")
print(f"\n픽셀 값 범위: {np_img.min()} ~ {np_img.max()}")
print(f"데이터 타입: {np_img.dtype}")
print(f"\n좌측 상단 3x3 영역 (Red 채널):\n{np_img[:3, :3, 0]}")

### 이미지 리사이즈와 크롭

PIL로 이미지 크기를 조절하고, numpy 슬라이싱으로 특정 영역을 잘라낼 수 있습니다.

In [ ]:
# PIL로 리사이즈
resized = img.resize((128, 128))
print(f"리사이즈: {np.array(img).shape} → {np.array(resized).shape}")

# numpy로 크롭 (좌상단 80x80 영역)
np_resized = np.array(resized)
cropped = np_resized[20:100, 24:104]
print(f"크롭: {np_resized.shape} → {cropped.shape}")
Image.fromarray(cropped)

### RGB 채널 분리와 그레이스케일

이미지는 Red, Green, Blue 3개 채널의 조합입니다.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# 상단: 원본 + 각 채널 히트맵
axes[0, 0].imshow(cropped)
axes[0, 0].set_title("Original")

for i, (name, cmap) in enumerate([("Red", "Reds"), ("Green", "Greens")]):
    axes[0, i + 1].imshow(cropped[:, :, i], cmap=cmap)
    axes[0, i + 1].set_title(f"{name} Channel (heatmap)")

# 하단: Blue 채널 + 그레이스케일 + 밝기 히스토그램
axes[1, 0].imshow(cropped[:, :, 2], cmap="Blues")
axes[1, 0].set_title("Blue Channel (heatmap)")

grey = np.mean(cropped, axis=2).astype(np.uint8)
axes[1, 1].imshow(grey, cmap="gray")
axes[1, 1].set_title(f"Grayscale {grey.shape}")

axes[1, 2].hist(grey.ravel(), bins=50, color="gray", alpha=0.7)
axes[1, 2].set_title("Pixel Intensity Histogram")
axes[1, 2].set_xlabel("Pixel Value")

for ax in axes.flat[:5]:
    ax.axis("off")
plt.tight_layout()
plt.show()

## 2. PyTorch 텐서 소개

PyTorch는 GPU 가속을 지원하는 텐서 연산 라이브러리입니다.
numpy 배열과 유사하지만, **자동 미분(autograd)** 기능이 핵심입니다.

In [ ]:
import torch

# 다양한 방법으로 텐서 생성
t1 = torch.zeros(2, 3)
t2 = torch.ones(2, 3)
t3 = torch.randn(2, 3)  # 표준 정규분포
print(f"zeros:\n{t1}")
print(f"\nrandn:\n{t3}")

# numpy 배열 → PyTorch 텐서
np_arr = np.array([[1.0, 2.0], [3.0, 4.0]])
tensor = torch.from_numpy(np_arr)
print(f"\nnumpy {np_arr.shape} → tensor {tensor.shape}")

# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n사용 디바이스: {device}")

## 3. 신경망의 가장 작은 단위: Linear 레이어

Linear 레이어는 다음 연산을 수행합니다:

$$y = xW^T + b$$

- **x**: 입력 (input_features 차원)
- **W**: 가중치 행렬 (output_features × input_features)
- **b**: 편향 벡터 (output_features 차원)
- **y**: 출력 (output_features 차원)

In [ ]:
import torch.nn as nn

# 6차원 입력 → 3차원 출력 Linear 레이어
linear = nn.Linear(6, 3, bias=True).to(device)

print(f"가중치(W) shape: {linear.weight.shape}  → 3×6 = 18개 파라미터")
print(f"편향(b) shape:   {linear.bias.shape}   → 3개 파라미터")
print(f"총 파라미터:     {sum(p.numel() for p in linear.parameters())}개")
print(f"\n초기 가중치:\n{linear.weight.data}")

## 4. 학습의 핵심 사이클

딥러닝 학습은 4단계의 반복입니다:

1. **Forward (순전파)**: 입력을 모델에 통과시켜 예측값을 계산
2. **Loss (손실 계산)**: 예측값과 정답의 차이를 측정
3. **Backward (역전파)**: 손실에 대한 각 가중치의 기울기를 계산
4. **Step (가중치 업데이트)**: 기울기 방향으로 가중치를 조정

In [ ]:
# 3-클래스 분류 문제 설정
inputs = np.array([1.0, 3.0, 5.0, 7.0, 9.0, 2.0])
target_class = 2  # 클래스 2로 분류되어야 하는 입력

X = torch.tensor(inputs, dtype=torch.float32).unsqueeze(0).to(device)  # (1, 6)
Y = torch.tensor([target_class]).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(linear.parameters(), lr=0.05)

print(f"입력 X: {X.data}, shape: {X.shape}")
print(f"정답 Y: {Y.item()} (클래스 {target_class})")

In [ ]:
# === 학습 1 사이클 상세 관찰 ===

# 1단계: Forward
optimizer.zero_grad()
prediction = linear(X)
print(f"[Forward] 예측 점수: {prediction.data.squeeze()}")
print(f"  → 예측 클래스: {prediction.argmax(dim=1).item()}, 정답: {target_class}")

# 2단계: Loss
loss = criterion(prediction, Y)
print(f"\n[Loss] 손실값: {loss.item():.4f}")

# 3단계: Backward
loss.backward()
print(f"\n[Backward] weight.grad (일부):\n{linear.weight.grad[:, :3]}")

# 4단계: Step
w_before = linear.weight.data.clone()
optimizer.step()
w_after = linear.weight.data
print(f"\n[Step] 가중치 변화량 (L2 norm): {(w_after - w_before).norm():.6f}")

## 5. 반복 학습으로 손실 줄이기

위의 4단계를 여러 번 반복하면, 가중치가 점차 최적값에 수렴하며 손실이 줄어듭니다.

In [ ]:
# 30 에포크 학습 + 정확도 변화 추적
losses = []
predictions_history = []

for epoch in range(30):
    optimizer.zero_grad()
    pred = linear(X)
    loss = criterion(pred, Y)
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
    probs = torch.softmax(pred, dim=1).detach().cpu().squeeze()
    predictions_history.append(probs.numpy())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:02d} | loss={loss.item():.6f} | "
              f"P(class0)={probs[0]:.3f} P(class1)={probs[1]:.3f} P(class2)={probs[2]:.3f}")

# 학습 과정 시각화: Loss + 클래스별 확률 변화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(losses, linewidth=2)
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("Loss Curve")
ax1.grid(True, alpha=0.3)

preds_arr = np.array(predictions_history)
for c in range(3):
    style = "-" if c == target_class else "--"
    ax2.plot(preds_arr[:, c], style, label=f"Class {c}", linewidth=2)
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Probability")
ax2.set_title("Class Probability over Training")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 정리

| 개념 | 설명 |
|---|---|
| 이미지 | 숫자로 이루어진 3차원 배열 (H × W × C) |
| 텐서 | PyTorch의 기본 데이터 구조 (GPU 연산 + 자동 미분 지원) |
| Linear 레이어 | y = xW^T + b 연산을 수행하는 신경망의 기본 단위 |
| 학습 사이클 | Forward → Loss → Backward → Step 의 반복 |

**다음 노트북에서는** 실제 이미지 데이터셋(FashionMNIST)을 사용하여 완전연결 신경망을 학습해봅니다.